In [4]:
#access to console.developers.google.com (Google Cloud Api) create a project -> click on that project -> Getting Started
#choose Explore and enable APIs -> chooose gg sheet api -> enable it-> choose credentials -> choose Manage 
#service accounts choose Create service account -> name it -> choose Basic-Editor right -> dowload key json 
#file to the same folder with your code
import requests
import json

#API on Tiki need authentication, so we need the HEADER below
#After tracking network on Tiki page, copy cUrl -> import by raw text to Postman
#Paste the link API to Postman try to send -> click Code button on the right hand -> choose generate code with Python 
#or any language on left hand tab -> copy that code to here

payload={}

headers = {
  'authority': 'tiki.vn',
  'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
  'accept': 'application/json, text/plain, */*',
  'x-guest-token': 'Please fill it with your token',
  'sec-ch-ua-mobile': '?0',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-mode': 'cors',
  'sec-fetch-dest': 'empty',
  'referer': 'https://tiki.vn/dien-thoai-smartphone/c1795?page=3&src=c.1789.hamburger_menu_fly_out_banner',
  'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
  'cookie': 'Please fill it with your cookie'
}

In [6]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
import gspread as gs
import pandas as pd

#key.json is the file dowloaded from creating credential on console.developers.google.com
SERVICE_ACCOUNT_FILE = 'key.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
# The ID and range of a sample spreadsheet (end point of created spreadsheet).
SAMPLE_SPREADSHEET_ID = '1d8v_I28CqxCnsKAgLr6AM5RAlPbI6qD8A4_-cRy9V5Q'

service = build('sheets', 'v4', credentials=creds)
# Call the Sheets API
sheet = service.spreadsheets()
#read data from spreadsheet -> print result to see data
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="BT2!A1:B2").execute()

# authorize the clientsheet 
gc = gs.authorize(creds)

#print(result)


#link to get data (smartphone category) on tiki
tempUrl = "https://tiki.vn/api/v2/products?limit=48&include=advertisement&aggregations=1&trackity_id=8d720949-9bc4\
-df8b-e314-418038cd38e0&category=1795&src=c.1789.hamburger_menu_fly_out_banner&urlKey=dien-thoai-smartphone&page="


In [10]:
#We have 3 pages from the link, loop through them to get data
def GetAllDataOnTikiPagesHotDeal():
    page = 1
    list_data = []
    for x in range(page, 4):
        url = tempUrl + str(page)
        #print(url)
        response = requests.request("GET", url, headers=headers, data=payload)
        data = response.json()

        # convert the json to dataframe
        tmp = data["data"]
        #print(len(tmp))
        list_data.extend(tmp)#concat the lists
        print(len(list_data))    
        page = page + 1
    return list_data
#call function to print the list and see how many item on every pages
#list_data = GetAllDataOnTikiPagesHotDeal()

In [11]:
#clean data step, get columns that we need
out_put = []

for i in list_data:
    #remove items which not a smartphone
    if "Ốp lưng".lower() not in i['name'].lower():
        tmp = {
            'id': i['id'],
            'name': i['name'],
            'brand_name': i['brand_name'],
            'short_description': i['short_description'],
            'price': i['price'],
            'list_price': i['list_price'],
            'discount': i['discount'],
            'discount_rate': i['discount_rate'],
            'inventory_status': i['inventory_status'],
            'is_visible': i['is_visible'],
            'rating_average': i['rating_average'],
            'review_count': i['review_count'],
            'stock_item': i['stock_item']['qty'],
            'max_sale_qty': i['stock_item']['max_sale_qty'],
            'seller_product_id': i['seller_product_id'],
        }
        out_put.append(tmp)
#let see first object
#out_put[0]


In [12]:
#look the data by DataFrame
df = pd.DataFrame(out_put)
#get 10 row limited
df.head(10)

,id,name,brand_name,short_description,price,list_price,discount,discount_rate,inventory_status,is_visible,rating_average,review_count,stock_item,max_sale_qty,seller_product_id
0,53827214,Điện Thoại Oppo A12 (3GB/32GB) - Hàng Chính Hãng,OPPO,Màn hình giọt nước tràn viền\nOPPO A12 vẫn sở ...,2390000,2990000,600000,20,available,True,4.7,174,2551,4,63471809
1,58065346,Điện Thoại Realme C11 (2GB/32GB) - Hàng Chính ...,Realme,\nTràn đầy năng lượng với pin “siêu khủng” 500...,2149000,2690000,541000,20,available,True,4.8,102,687,4,58065348
2,70771651,Điện Thoại iPhone 12 Pro Max 128GB - Hàng Chín...,Apple,iPhone 12 Pro Max 128GB - Nâng tầm đẳng cấp sử...,29950000,33990000,4040000,12,available,True,5.0,435,12,20,70771653
3,63839533,Điện Thoại Vsmart Live 4 (6GB/64GB) - Hàng Chí...,Vsmart,"\nThiết kế sang trọng, tinh tế\nĐiện thoại Vsm...",3699000,4790000,1091000,23,available,True,4.8,5,50,20,73575051
4,45339109,Điện Thoại Samsung Galaxy A51 (6GB/128GB) - Hà...,Samsung,\nBắt trọn đường nét mọi tuyệt tác\nĐiện Thoại...,5790000,7990000,2200000,28,available,True,4.6,256,149,2,58805672
5,67910637,Điện Thoại Vsmart Joy 4 - Hàng Chính Hãng,Vsmart,"\nThiết kế sang trọng, tinh tế\nTiếp nối sự th...",2790000,3290000,500000,15,available,True,4.7,1446,1404,20,67910639
6,58259141,Điện Thoại Samsung Galaxy M31 (6GB/128GB) - Hà...,Samsung,\nMở Rộng Không Gian Trải Nghiệm Với Màn Hình ...,4959000,6490000,1531000,24,available,True,4.7,1715,41,2,58259145
7,74240829,Điện thoại Xiaomi POCO M3 - HÀNG CHÍNH HÃNG,Xiaomi,"Tuy nằm ở phân khúc giá rẻ, nhưng Xiaomi POCO ...",3209000,3990000,781000,20,available,True,4.8,36,27,2,83366458
8,32033717,Điện Thoại iPhone 11 64GB - Hàng Chính Hãng,Apple,\nThiết kế đậm chất phá cách nhưng cũng không ...,16290000,22990000,6700000,29,available,True,4.8,615,4,20,32033723
9,52717588,Điện Thoại Samsung Galaxy M11 (3GB/32GB) - Hàn...,Samsung,Trải Nghiệm Màn Hình Infinity-O Ấn Tượng\nĐắm ...,2990000,3690000,700000,19,available,True,4.7,1006,2657,2,52717590


In [18]:
#get by using open_by_key will work and by name is not
sh = gc.open_by_key('1d8v_I28CqxCnsKAgLr6AM5RAlPbI6qD8A4_-cRy9V5Q')

worksheet = sh.get_worksheet(2)
worksheet.update([df.columns.values.tolist()] + df.values.tolist())
